In [1]:
import gym
import holdem
import agent

Using TensorFlow backend.


In [2]:
def add_users(env):
    model_list = list()

    # start with 4 players
    env.add_player(0, stack=5000) # add a player to seat 0 with 1000 "chips"
#     model_list.append(agent.allFoldModel())
    model_list.append(agent.MontecarloModel())

    env.add_player(1, stack=5000) # add another player to seat 1 with 2000 "chips"
#     model_list.append(agent.allFoldModel())
    model_list.append(agent.MontecarloModel())

    env.add_player(2, stack=5000) # add another player to seat 2 with 3000 "chips"
    # model_list.append(agent.allFoldModel())
    model_list.append(agent.MontecarloModel())

    env.add_player(3, stack=5000) # add another player to seat 3 with 1000 "chips"
    model_list.append(agent.allFoldModel())

    env.add_player(4, stack=5000) # add another player to seat 3 with 1000 "chips"
    model_list.append(agent.dqnModel())

    env.add_player(5, stack=5000) # add another player to seat 3 with 1000 "chips"
    model_list.append(agent.dqnModel())
#     model_list.append(agent.allFoldModel())

    # env.add_player(6, stack=1000) # add another player to seat 3 with 1000 "chips"
    # model_list.append(agent.allCallModel())

    # env.add_player(7, stack=1000) # add another player to seat 3 with 1000 "chips"
    # model_list.append(agent.MontecarloModel())

    # env.add_player(8, stack=1000) # add another player to seat 3 with 1000 "chips"
    # model_list.append(agent.allFoldModel())

    # env.add_player(9, stack=1000) # add another player to seat 3 with 1000 "chips"
    # model_list.append(agent.allinModel())

    return model_list

In [3]:
playerid = 4
episodes = 30
pre_train_steps = 20 #How many steps of random actions before training begins.

def lets_play():
    env = gym.make('TexasHoldem-v2') # holdem.TexasHoldemEnv(2)

    model_list = add_users(env)
    
    agent = model_list[playerid]
    total_steps = 0

    # Iterate the game
    for e in range(episodes):
    #     lets_play(env, env.n_seats, model_list)
        # reset state in the beginning of each game
        state = env.reset()
        env.render(mode='human')
        done = False
        if env.episode_end:
            break
        stack = state.player_states[playerid].stack
        # time_t represents each frame of the game
        # Our goal is to keep the pole upright as long as possible until score of 500
        # the more time_t the more score
        while not done:
            # play safe actions, check when no one else has raised, call when raised.
            # print(">>> Debug Information ")
            # print("state(t)")
            # for p in cur_state.player_states:
            #     print(p)
            # print(cur_state.community_state)

            actions = holdem.model_list_action(state, n_seats=env.n_seats, model_list=model_list)
            next_state, rewards, done, _ = env.step(actions)
            action = actions[playerid][0]
            reward = rewards[playerid] - stack
            stack = rewards[playerid]
            if state.community_state.current_player == playerid:
                agent.remember(state, action, reward, next_state, done, playerid)
#                 print('actions: ', action)
                agent.onlineTrainModel()
                env.render(mode='human')
            state = next_state
            #print("memory len: ", len(agent.memory))

    #         print('table card: ', state.community_card)

            # print("action(t), (CALL=1, RAISE=2, FOLD=3 , CHECK=0, [action, amount])")
            # print(actions)
            if done:
                # print the score and break out of the loop
                print("episode: {}/{}, score: {}".format(e, episodes, reward))

            # print("reward(t+1)")
            # print(rews)
            # print("<<< Debug Information ")

            # print("final state")
            # print(cur_state)

            # total_stack = sum([p.stack for p in env._seats])
            # if total_stack != 10000:
            #     return

            # train the agent with the experience of the episode
        if total_steps > pre_train_steps:
            agent.replay(10)

        total_steps += 1

        agent.saveModel()

        print("Episode End!!!")



In [5]:
for i in range(30):
    print(i, " plays")
    try:
        lets_play()
    except:
        print('Error!!!!')

0  plays
Cycle 1, total pot: 30 >>>
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [5♥],[J♦] stack: 5000
1 [3♣],[Q♠] stack: 4990
2 [4♦],[7♣] stack: 4980
3 [2♠],[K♠] stack: 5000
4 [8♠],[2♦] stack: 5000
5 [3♠],[6♣] stack: 5000
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
actions:  2
Cycle 1, total pot: 1050 >>>
last action by player 4:	^ raise, current bet: 1020
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [5♥],[J♦] stack: 5000
1 [3♣],[Q♠] stack: 4990
2 [4♦],[7♣] stack: 4980
3 [2♠],[K♠] stack: 5000
4 [8♠],[2♦] stack: 3980
5 [3♠],[6♣] stack: 5000
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
actions:  0
Cycle 1, total pot: 4080 >>>
last action by player 4:	_ check
community:
- [8♦],[7♠],[7♦],[  ],[  ] 
players:
0 [5♥],[J♦] stack: 3980
1 [3♣],[Q♠] stack: 3980
2 [4♦],[7♣] stack: 3980
3 [2♠],[K♠] stack: 5000
4 [8♠],[2♦] stack: 3980
5 [3♠],[6♣] stack: 5000
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [

Cycle 1, total pot: 30 >>>
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [9♣],[9♦] stack: 5000
1 [3♣],[J♠] stack: 4990
2 [2♥],[8♥] stack: 4980
3 [4♣],[8♠] stack: 5000
4 [K♥],[4♦] stack: 5000
5 [J♣],[6♦] stack: 5000
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
actions:  3
Cycle 1, total pot: 30 >>>
last action by player 4:	x fold
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [9♣],[9♦] stack: 5000
1 [3♣],[J♠] stack: 4990
2 [2♥],[8♥] stack: 4980
3 [4♣],[8♠] stack: 5000
4 [K♥],[4♦] stack: 5000
5 [J♣],[6♦] stack: 5000
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
episode: 0/30, score: 0
Episode End!!!
Cycle 2, total pot: 30 >>>
last action by player 5:	x fold
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [4♥],[K♥] stack: 8060
1 [A♦],[3♥] stack: 3980
2 [Q♥],[5♦] stack: 3970
3 [9♥],[4♠] stack: 4980
4 [Q♦],[K♠] stack: 5000
5 [2♦],[J♣] stack: 3980
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  

Episode End!!!
Cycle 2, total pot: 0 >>>
last action by player 4:	_ check
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [2♥],[Q♠] stack: 4980
1 [4♦],[7♣] stack: 4980
2 [5♦],[K♦] stack: 4980
3 [2♦],[5♥] stack: 5000
4 [K♠],[3♥] stack: 10060
5 [7♠],[Q♦] stack: 0
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
6  plays
Cycle 1, total pot: 30 >>>
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [4♦],[6♠] stack: 5000
1 [2♠],[A♠] stack: 4990
2 [6♥],[6♣] stack: 4980
3 [3♠],[K♦] stack: 5000
4 [J♦],[T♠] stack: 5000
5 [4♥],[K♣] stack: 5000
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
actions:  3
Cycle 1, total pot: 30 >>>
last action by player 4:	x fold
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [4♦],[6♠] stack: 5000
1 [2♠],[A♠] stack: 4990
2 [6♥],[6♣] stack: 4980
3 [3♠],[K♦] stack: 5000
4 [J♦],[T♠] stack: 5000
5 [4♥],[K♣] stack: 5000
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ]

actions:  0
Cycle 3, total pot: 110 >>>
last action by player 4:	_ check
community:
- [A♦],[J♦],[9♠],[K♠],[  ] 
players:
0 [3♥],[J♣] stack: 3940
1 [3♠],[T♥] stack: 3940
2 [4♦],[3♣] stack: 3940
3 [7♣],[8♦] stack: 4970
4 [6♣],[8♥] stack: 3960
5 [2♦],[7♦] stack: 9140
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
actions:  0
Cycle 3, total pot: 110 >>>
last action by player 4:	_ check
community:
- [A♦],[J♦],[9♠],[K♠],[  ] 
players:
0 [3♥],[J♣] stack: 3940
1 [3♠],[T♥] stack: 3940
2 [4♦],[3♣] stack: 3940
3 [7♣],[8♦] stack: 4970
4 [6♣],[8♥] stack: 3960
5 [2♦],[7♦] stack: 9140
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
actions:  0
Cycle 3, total pot: 110 >>>
last action by player 4:	_ check
community:
- [A♦],[J♦],[9♠],[K♠],[5♣] 
players:
0 [3♥],[J♣] stack: 3940
1 [3♠],[T♥] stack: 3940
2 [4♦],[3♣] stack: 3940
3 [7♣],[8♦] stack: 4970
4 [6♣],[8♥] stack: 3960
5 [2♦],[7♦] stack: 9140
6 [  ],[  ] stack: 0
7 [  ],[

Cycle 1, total pot: 30 >>>
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [K♦],[7♠] stack: 5000
1 [K♣],[K♥] stack: 4990
2 [8♠],[4♠] stack: 4980
3 [8♥],[6♦] stack: 5000
4 [T♠],[J♠] stack: 5000
5 [2♥],[7♦] stack: 5000
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
actions:  2
Cycle 1, total pot: 5030 >>>
last action by player 4:	^ raise, current bet: 5000
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [K♦],[7♠] stack: 5000
1 [K♣],[K♥] stack: 4990
2 [8♠],[4♠] stack: 4980
3 [8♥],[6♦] stack: 5000
4 [T♠],[J♠] stack: 0
5 [2♥],[7♦] stack: 5000
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
Error!!!!
12  plays
Cycle 1, total pot: 30 >>>
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [T♥],[5♣] stack: 5000
1 [9♣],[2♣] stack: 4990
2 [J♦],[9♥] stack: 4980
3 [A♦],[Q♦] stack: 5000
4 [3♣],[9♠] stack: 5000
5 [8♦],[6♠] stack: 5000
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0


Cycle 1, total pot: 30 >>>
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [5♣],[4♠] stack: 5000
1 [2♣],[3♠] stack: 4990
2 [K♥],[9♦] stack: 4980
3 [5♠],[3♣] stack: 5000
4 [3♦],[2♦] stack: 5000
5 [J♣],[5♥] stack: 5000
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
actions:  1
Cycle 1, total pot: 50 >>>
last action by player 4:	- call, current bet: 20
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [5♣],[4♠] stack: 5000
1 [2♣],[3♠] stack: 4990
2 [K♥],[9♦] stack: 4980
3 [5♠],[3♣] stack: 5000
4 [3♦],[2♦] stack: 4980
5 [J♣],[5♥] stack: 5000
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
actions:  2
Cycle 1, total pot: 100 >>>
last action by player 4:	^ raise, current bet: 4980
community:
- [9♠],[8♥],[J♥],[  ],[  ] 
players:
0 [5♣],[4♠] stack: 4980
1 [2♣],[3♠] stack: 4980
2 [K♥],[9♦] stack: 4980
3 [5♠],[3♣] stack: 5000
4 [3♦],[2♦] stack: 4980
5 [J♣],[5♥] stack: 4980
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0

Episode End!!!
Cycle 2, total pot: 0 >>>
last action by player 1:	_ check
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [2♥],[5♥] stack: 3980
1 [4♠],[J♦] stack: 0
2 [7♣],[9♠] stack: 3980
3 [3♠],[2♠] stack: 5000
4 [A♦],[6♣] stack: 4980
5 [3♦],[Q♦] stack: 12060
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
20  plays
Cycle 1, total pot: 30 >>>
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [Q♦],[9♣] stack: 5000
1 [J♣],[9♦] stack: 4990
2 [3♠],[4♣] stack: 4980
3 [4♠],[2♥] stack: 5000
4 [T♥],[7♠] stack: 5000
5 [Q♠],[7♦] stack: 5000
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
actions:  1
Cycle 1, total pot: 50 >>>
last action by player 4:	- call, current bet: 20
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [Q♦],[9♣] stack: 5000
1 [J♣],[9♦] stack: 4990
2 [3♠],[4♣] stack: 4980
3 [4♠],[2♥] stack: 5000
4 [T♥],[7♠] stack: 4980
5 [Q♠],[7♦] stack: 5000
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[ 

Cycle 1, total pot: 30 >>>
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [7♣],[T♥] stack: 5000
1 [J♥],[A♠] stack: 4990
2 [Q♠],[J♦] stack: 4980
3 [6♦],[3♥] stack: 5000
4 [8♥],[T♣] stack: 5000
5 [6♥],[3♠] stack: 5000
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
actions:  2
Cycle 1, total pot: 1050 >>>
last action by player 4:	^ raise, current bet: 1020
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [7♣],[T♥] stack: 5000
1 [J♥],[A♠] stack: 4990
2 [Q♠],[J♦] stack: 4980
3 [6♦],[3♥] stack: 5000
4 [8♥],[T♣] stack: 3980
5 [6♥],[3♠] stack: 5000
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
Error!!!!
24  plays
Cycle 1, total pot: 30 >>>
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [Q♥],[3♠] stack: 5000
1 [A♣],[Q♦] stack: 4990
2 [2♠],[5♣] stack: 4980
3 [3♣],[7♣] stack: 5000
4 [J♣],[9♦] stack: 5000
5 [A♦],[6♦] stack: 5000
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack:

Cycle 1, total pot: 30 >>>
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [5♦],[K♦] stack: 5000
1 [6♣],[K♠] stack: 4990
2 [Q♠],[2♦] stack: 4980
3 [3♦],[J♠] stack: 5000
4 [K♥],[2♠] stack: 5000
5 [2♥],[4♦] stack: 5000
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
actions:  2
Cycle 1, total pot: 5030 >>>
last action by player 4:	^ raise, current bet: 5000
community:
- [  ],[  ],[  ],[  ],[  ] 
players:
0 [5♦],[K♦] stack: 5000
1 [6♣],[K♠] stack: 4990
2 [Q♠],[2♦] stack: 4980
3 [3♦],[J♠] stack: 5000
4 [K♥],[2♠] stack: 0
5 [2♥],[4♦] stack: 5000
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  ],[  ] stack: 0
<<<
actions:  0
Cycle 1, total pot: 20000 >>>
last action by player 4:	_ check
community:
- [T♣],[8♦],[A♥],[  ],[  ] 
players:
0 [5♦],[K♦] stack: 0
1 [6♣],[K♠] stack: 0
2 [Q♠],[2♦] stack: 0
3 [3♦],[J♠] stack: 5000
4 [K♥],[2♠] stack: 0
5 [2♥],[4♦] stack: 5000
6 [  ],[  ] stack: 0
7 [  ],[  ] stack: 0
8 [  ],[  ] stack: 0
9 [  